In [323]:
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.offline as offline
import plotly.plotly as py
import pandas as pd
import numpy as np


In [327]:
df_airports = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_february_us_airport_traffic.csv')
df_airports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 8 columns):
iata       221 non-null object
airport    221 non-null object
city       220 non-null object
state      220 non-null object
country    221 non-null object
lat        221 non-null float64
long       221 non-null float64
cnt        221 non-null int64
dtypes: float64(2), int64(1), object(5)
memory usage: 13.9+ KB


In [12]:
df_flight_paths = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_february_aa_flight_paths.csv')
df_flight_paths.head()

,start_lat,start_lon,end_lat,end_lon,airline,airport1,airport2,cnt
0,32.895951,-97.037200,35.040222,-106.609194,AA,DFW,ABQ,444
1,41.979595,-87.904464,30.194533,-97.669872,AA,ORD,AUS,166
2,32.895951,-97.037200,41.938874,-72.683228,AA,DFW,BDL,162
3,18.439417,-66.001833,41.938874,-72.683228,AA,SJU,BDL,56
4,32.895951,-97.037200,33.562943,-86.753550,AA,DFW,BHM,168


In [62]:
df_flight_paths[df_flight_paths['airport1']=='BDL']

,start_lat,start_lon,end_lat,end_lon,airline,airport1,airport2,cnt
104,41.938874,-72.683228,25.79325,-80.290556,AA,BDL,MIA,112


In [63]:
df_flight_paths[df_flight_paths['airport1']=='MIA']

,start_lat,start_lon,end_lat,end_lon,airline,airport1,airport2,cnt
5,25.79325,-80.290556,36.124477,-86.678182,AA,MIA,BNA,56
7,25.79325,-80.290556,42.364348,-71.005179,AA,MIA,BOS,392
11,25.79325,-80.290556,39.175402,-76.668198,AA,MIA,BWI,112
17,25.79325,-80.290556,39.858408,-104.667002,AA,MIA,DEN,112
38,25.79325,-80.290556,32.895951,-97.037200,AA,MIA,DFW,560
57,25.79325,-80.290556,42.212059,-83.348836,AA,MIA,DTW,112
62,25.79325,-80.290556,40.692497,-74.168661,AA,MIA,EWR,168
69,25.79325,-80.290556,38.944532,-77.455810,AA,MIA,IAD,112
70,25.79325,-80.290556,29.980472,-95.339722,AA,MIA,IAH,168
79,25.79325,-80.290556,40.639751,-73.778926,AA,MIA,JFK,392


In [52]:
df_country_lon_lat = pd.read_csv('country_lon_lat.csv')
df_country_lon_lat[df_country_lon_lat['name'] == 'Namibia']

,country,latitude,longitude,name
156,NaN,-22.95764,18.49041,Namibia


In [41]:
df_country = pd.read_csv('country.csv')
df_country.head()

,Code Name
0,AD ANDORRA
1,AE UNITED ARAB EMIRATES
2,AF AFGHANISTAN
3,AG ANTIGUA AND BARBUDA
4,AI ANGUILLA


In [42]:
df_country['Code'], df_country['Name'] = df_country['Code Name'].str.split(' ', 1).str
df_country = df_country.drop(['Code Name'],1)

In [43]:
df_country = df_country.rename(index=str, columns={"Code": "country"})

In [46]:
df_join_count = df_country.join(df_country_lon_lat.set_index('country'), on='country')

In [55]:
df_join_count.to_csv('joined_country_lat_lon.csv')

In [304]:
df_ref = pd.read_csv('joined_country_lat_lon.csv')
df_ref.head()

,Unnamed: 0,country,Name,latitude,longitude,name
0,0,AD,ANDORRA,42.546245,1.601554,Andorra
1,1,AE,UNITED ARAB EMIRATES,23.424076,53.847818,United Arab Emirates
2,2,AF,AFGHANISTAN,33.939110,67.709953,Afghanistan
3,3,AG,ANTIGUA AND BARBUDA,17.060816,-61.796428,Antigua and Barbuda
4,4,AI,ANGUILLA,18.220554,-63.068615,Anguilla


In [56]:
df_new = pd.read_csv('trade_complete.csv')

In [222]:
df_trade = df_new[['Year','Importer','Exporter','Unnamed: 0']]

In [223]:
df_trade.head()

,Year,Importer,Exporter,Unnamed: 0
0,1975.0,CH,YU,0
1,1975.0,CH,DE,1
2,1975.0,CH,DE,2
3,1975.0,CH,DE,3
4,1975.0,CA,DE,4


In [224]:
df_trade = df_trade.dropna()

In [195]:
df_1975 = df_trade[df_trade['Year']==1975]

In [227]:
def trade_count(df):
    for i in range(len(df)):
        pair = '-'.join(s for s in sorted([df['Importer'].iloc[i],df['Exporter'].iloc[i]]))
        df.loc[i,'pair'] = pair
    return df.groupby(['Year', 'pair'],as_index=False).sum()

In [225]:
df_groupby = df_trade.groupby(['Year','Importer','Exporter'],as_index=False).count()
df_groupby

,Year,Importer,Exporter,Unnamed: 0
0,1975.0,BE,CA,2
1,1975.0,BM,CR,1
2,1975.0,CA,DE,4
3,1975.0,CA,DK,1
4,1975.0,CA,GB,3
5,1975.0,CA,NZ,1
6,1975.0,CA,US,3
7,1975.0,CD,CA,1
8,1975.0,CH,BE,2
9,1975.0,CH,BW,1


In [228]:
df_pair = trade_count(df_groupby)
df_pair.head(20)

,Year,pair,Unnamed: 0
0,1975.0,BE-CA,2
1,1975.0,BE-CH,2
2,1975.0,BM-CR,1
3,1975.0,BW-CH,1
4,1975.0,CA-CD,1
5,1975.0,CA-CH,3
6,1975.0,CA-DE,7
7,1975.0,CA-DK,1
8,1975.0,CA-GB,5
9,1975.0,CA-JP,2


In [254]:
df_pair.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90431 entries, 0 to 90430
Data columns (total 3 columns):
Year          90431 non-null float64
pair          90431 non-null object
Unnamed: 0    90431 non-null int64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.8+ MB


In [231]:
df_split = pd.DataFrame(df_pair.pair.str.split('-',1).tolist(),columns = ['P1','P2'])
df_split.head()

,P1,P2
0,BE,CA
1,BE,CH
2,BM,CR
3,BW,CH
4,CA,CD


In [232]:
df_concat = pd.concat([df_pair, df_split], axis = 1)
df_concat.head()

,Year,pair,Unnamed: 0,P1,P2
0,1975.0,BE-CA,2,BE,CA
1,1975.0,BE-CH,2,BE,CH
2,1975.0,BM-CR,1,BM,CR
3,1975.0,BW-CH,1,BW,CH
4,1975.0,CA-CD,1,CA,CD


In [233]:
df_concat = df_concat.rename(index=str, columns={"Unnamed: 0": "Vol"})

In [259]:
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
Index: 90431 entries, 0 to 90430
Data columns (total 5 columns):
Year    90431 non-null float64
pair    90431 non-null object
Vol     90431 non-null int64
P1      90431 non-null object
P2      90431 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 4.1+ MB


In [283]:
df_clean = df_concat.drop(df_concat[df_concat['P1']=='XX'].index)
df_clean = df_clean.drop(df_concat[df_concat['P2']=='XX'].index)

In [284]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 89312 entries, 0 to 90430
Data columns (total 5 columns):
Year    89312 non-null float64
pair    89312 non-null object
Vol     89312 non-null int64
P1      89312 non-null object
P2      89312 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 4.1+ MB


In [285]:
df_clean = df_clean.drop(df_concat[df_concat['P2']=='XV'].index)
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 89249 entries, 0 to 90430
Data columns (total 5 columns):
Year    89249 non-null float64
pair    89249 non-null object
Vol     89249 non-null int64
P1      89249 non-null object
P2      89249 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 4.1+ MB


In [286]:
df_clean = df_clean.drop(df_concat[df_concat['P2']=='ZZ'].index)

In [305]:
df_clean = df_clean.rename(index=str, columns={"P1": "country"})

In [308]:
df_edge = df_clean.merge(df_ref, on='country', how='left')

In [310]:
df_edge = df_edge[['Year','pair','Vol','country','P2','latitude','longitude']]
df_edge = df_edge.rename(index=str, columns={'country':'country1','P2':'country','latitude':'lat1',\
                                             'longitude':'long1'})
df_edge.head()

,Year,pair,Vol,country1,country,lat1,long1
0,1975.0,BE-CA,2,BE,CA,50.503887,4.469936
1,1975.0,BE-CH,2,BE,CH,50.503887,4.469936
2,1975.0,BM-CR,1,BM,CR,32.321384,-64.757370
3,1975.0,BW-CH,1,BW,CH,-22.328474,24.684866
4,1975.0,CA-CD,1,CA,CD,56.130366,-106.346771


In [314]:
df_edge = df_edge.merge(df_ref, on='country', how='left')
df_edge_2 = df_edge[['Year','pair','Vol','country1','country','lat1','long1','latitude','longitude']]
df_edge_2 = df_edge_2.rename(index=str, columns={'country':'country2','latitude':'lat2',\
                                             'longitude':'long2'})

In [315]:
df_edge_2.head()

,Year,pair,Vol,country1,country2,lat1,long1,lat2,long2
0,1975.0,BE-CA,2,BE,CA,50.503887,4.469936,56.130366,-106.346771
1,1975.0,BE-CH,2,BE,CH,50.503887,4.469936,46.818188,8.227512
2,1975.0,BM-CR,1,BM,CR,32.321384,-64.757370,9.748917,-83.753428
3,1975.0,BW-CH,1,BW,CH,-22.328474,24.684866,46.818188,8.227512
4,1975.0,CA-CD,1,CA,CD,56.130366,-106.346771,-4.038333,21.758664


In [332]:
df_ref['name']

0                               Andorra
1                  United Arab Emirates
2                           Afghanistan
3                   Antigua and Barbuda
4                              Anguilla
5                               Albania
6                               Armenia
7                  Netherlands Antilles
8                                Angola
9                            Antarctica
10                            Argentina
11                       American Samoa
12                              Austria
13                            Australia
14                                Aruba
15                        Aland Islands
16                           Azerbaijan
17               Bosnia and Herzegovina
18                             Barbados
19                           Bangladesh
20                              Belgium
21                         Burkina Faso
22                             Bulgaria
23                              Bahrain
24                              Burundi


In [316]:
df_edge_1975 = df_edge_2[df_edge_2['Year']==1975]

In [357]:
df_edge_1985 = df_edge_2[df_edge_2['Year']==1985]

In [358]:
df_edge_1995 = df_edge_2[df_edge_2['Year']==1995]

In [359]:
df_edge_2005 = df_edge_2[df_edge_2['Year']==2005]

In [360]:
df_edge_2015 = df_edge_2[df_edge_2['Year']==2015]

In [364]:
country = [ dict(
        type = 'scattergeo',
        lon = df_ref['longitude'],
        lat = df_ref['latitude'],
        hoverinfo = 'text',
        text = df_ref['Name'],
        mode = 'markers',
        marker = dict( 
            size=2, 
            color='rgb(255, 0, 0)',
            line = dict(
                width=3,
                color='rgba(68, 68, 68, 0)'
            )
        ))]
        
edge_paths = []
for i in range( len( df_edge_2015 ) ):
    edge_paths.append(
        dict(
            type = 'scattergeo',
            lon = [ df_edge_2015['long1'][i], df_edge_2015['long2'][i] ],
            lat = [ df_edge_2015['lat1'][i], df_edge_2015['lat2'][i] ],
            mode = 'lines',
            line = dict(
                width = 1,
                color = 'red',
            ),
            opacity = float(df_edge_2015['Vol'][i])/float(df_edge_2015['Vol'].max()),
        )
    )
    
layout = dict(
        title = '2015 Animal Trade Network',
        showlegend = False, 
        geo = dict(
            projection=dict( type='equirectangular' ),
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)',
        ),
    )
    
fig = dict( data=edge_paths + country, layout=layout )
offline.iplot( fig, filename='2015-trade')

In [334]:
df_list = []
y = range(1975, 2016)
for i in y:
    df_list.append(df_edge_2.loc[df_edge_2['Year'] == i])

In [354]:
years = ['1975','1976']

country = [ dict(
        type = 'scattergeo',
        lon = df_ref['longitude'],
        lat = df_ref['latitude'],
        hoverinfo = 'text',
        text = df_ref['Name'],
        mode = 'markers',
        marker = dict( 
            size=2, 
            color='rgb(255, 0, 0)',
            line = dict(
                width=3,
                color='rgba(68, 68, 68, 0)'
            )
        ))]

year = 1975
edge_paths = []
dataset_by_year = df_edge_2[df_edge_2['Year'] == year]
for i in range( len( dataset_by_year ) ):
    edge_paths.append(
        dict(
            type = 'scattergeo',
            lon = [ dataset_by_year['long1'][i], dataset_by_year['long2'][i] ],
            lat = [ dataset_by_year['lat1'][i], dataset_by_year['lat2'][i] ],
            mode = 'lines',
            line = dict(
                width = 1,
                color = 'red',
            ),
            opacity = float(dataset_by_year['Vol'][i])/float(dataset_by_year['Vol'].max()),
        )
    )
    
for year in years:
    frame = []
#     frame = {'data': [], 'name': str(year)}
    dataset_by_year = df_edge_2[df_edge_2['Year'] == int(year)]
    for i in range( len( dataset_by_year ) ):
        frame.append(
            dict(
                type = 'scattergeo',
                lon = [ dataset_by_year['long1'][i], dataset_by_year['long2'][i] ],
                lat = [ dataset_by_year['lat1'][i], dataset_by_year['lat2'][i] ],
                mode = 'lines',
                line = dict(
                    width = 1,
                    color = 'red',
                ),
#                 opacity = float(dataset_by_year['Vol'][i])/float(dataset_by_year['Vol'].max()),
            )
#             +
#             dict(
#                 type = 'scattergeo',
#                 lon = df_ref['longitude'],
#                 lat = df_ref['latitude'],
#                 hoverinfo = 'text',
#                 text = df_ref['Name'],
#                 mode = 'markers',
#                 marker = dict( 
#                     size=2, 
#                     color='rgb(255, 0, 0)',
#                     line = dict(
#                         width=3,
#                         color='rgba(68, 68, 68, 0)'
#                         )
#                     )
#                 )
            )
        
layout = dict(
        title = 'Animal Trade Network<br>(Hover for country names)',
        showlegend = False, 
        geo = dict(
            projection=dict( type='equirectangular' ),
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)',
        ),
    )
    
fig = dict( data=edge_paths + country, layout=layout, frames = frame)
offline.iplot( fig, filename='animal-trade',validate=False )

In [128]:
df_trade2 = df_new[['Year','Exporter','Importer']]
df_trade2 = df_trade2.dropna()
df_trade2 = df_trade2.rename(index=str, columns={"Exporter":"Importer","Importer":"Exporter"})
df_trade2.head()

,Year,Importer,Exporter
0,1975.0,YU,CH
1,1975.0,DE,CH
2,1975.0,DE,CH
3,1975.0,DE,CH
4,1975.0,DE,CA


In [129]:
df_trade3 = pd.concat([df_trade, df_trade2])

In [130]:
df_trade3[df_trade3['Importer'] == 'DE']

,Year,Importer,Exporter
126,1975.0,DE,CA
136,1975.0,DE,CA
139,1975.0,DE,CA
295,1976.0,DE,KE
300,1976.0,DE,KE
315,1976.0,DE,GB
333,1976.0,DE,GB
334,1976.0,DE,GB
335,1976.0,DE,GB
336,1976.0,DE,GB


In [131]:
df_trade3['Partner'] = df_trade3['Importer']+" "+df_trade3['Exporter']

In [145]:
df_partner = df_trade3.groupby(['Year','Partner'],as_index=False).count()

In [133]:
df_partner[df_partner['Partner']=='CA DE']

,Year,Partner,Importer,Exporter
7,1975.0,CA DE,7,7
109,1976.0,CA DE,9,9
534,1977.0,CA DE,35,35
1414,1978.0,CA DE,59,59
2706,1979.0,CA DE,62,62
4191,1980.0,CA DE,57,57
5969,1981.0,CA DE,101,101
8152,1982.0,CA DE,122,122
10593,1983.0,CA DE,96,96
13558,1984.0,CA DE,114,114


In [134]:
df_partner[df_partner['Partner']=='DE CA']

,Year,Partner,Importer,Exporter
41,1975.0,DE CA,7,7
182,1976.0,DE CA,9,9
663,1977.0,DE CA,35,35
1566,1978.0,DE CA,59,59
2901,1979.0,DE CA,62,62
4413,1980.0,DE CA,57,57
6244,1981.0,DE CA,101,101
8435,1982.0,DE CA,122,122
10979,1983.0,DE CA,96,96
13957,1984.0,DE CA,114,114


In [135]:
part_list = df_partner['Partner'].unique()

18741


In [146]:
df_partner_plit = pd.DataFrame(df_partner.Partner.str.split(' ',1).tolist(),columns = ['P1','P2'])

In [147]:
df_concat = pd.concat([df_partner, df_partner_plit], axis = 1)

In [189]:
df_con1975 = df_concat[df_concat['Year']==1975]
df_con1975

,Year,Partner,Importer,Exporter,P1,P2
0,1975.0,BE CA,2,2,BE,CA
1,1975.0,BE CH,2,2,BE,CH
2,1975.0,BM CR,1,1,BM,CR
3,1975.0,BW CH,1,1,BW,CH
4,1975.0,CA BE,2,2,CA,BE
5,1975.0,CA CD,1,1,CA,CD
6,1975.0,CA CH,3,3,CA,CH
7,1975.0,CA DE,7,7,CA,DE
8,1975.0,CA DK,1,1,CA,DK
9,1975.0,CA GB,5,5,CA,GB


In [154]:
df_con1975['P1'][0] + " " + df_con1975['P2'][0]

'BE CA'

In [172]:
df_con1975['P2'][0] + " " + df_con1975['P1'][0] in df_con1975['Partner'].values

True

In [170]:
df_con1975.Partner.values

array(['BE CA', 'BE CH', 'BM CR', 'BW CH', 'CA BE', 'CA CD', 'CA CH',
       'CA DE', 'CA DK', 'CA GB', 'CA JP', 'CA NZ', 'CA US', 'CD CA',
       'CH BE', 'CH BW', 'CH CA', 'CH DE', 'CH FI', 'CH FR', 'CH GB',
       'CH GH', 'CH GR', 'CH IN', 'CH IT', 'CH KE', 'CH LR', 'CH NC',
       'CH NI', 'CH NL', 'CH PG', 'CH PH', 'CH PT', 'CH PY', 'CH SC',
       'CH TH', 'CH US', 'CH XX', 'CH YU', 'CH ZA', 'CR BM', 'DE CA',
       'DE CH', 'DK CA', 'FI CH', 'FR CH', 'GB CA', 'GB CH', 'GB SE',
       'GH CH', 'GR CH', 'IN CH', 'IT CH', 'JP CA', 'KE CH', 'LR CH',
       'NC CH', 'NI CH', 'NL CH', 'NZ CA', 'PG CH', 'PH CH', 'PT CH',
       'PY CH', 'SC CH', 'SE GB', 'TH CH', 'US CA', 'US CH', 'XX CH',
       'YU CH', 'ZA CH'], dtype=object)

In [193]:
df_con1975.drop(df_con1975.index[[0]])

,Year,Partner,Importer,Exporter,P1,P2
1,1975.0,BE CH,2,2,BE,CH
2,1975.0,BM CR,1,1,BM,CR
3,1975.0,BW CH,1,1,BW,CH
4,1975.0,CA BE,2,2,CA,BE
5,1975.0,CA CD,1,1,CA,CD
6,1975.0,CA CH,3,3,CA,CH
7,1975.0,CA DE,7,7,CA,DE
8,1975.0,CA DK,1,1,CA,DK
9,1975.0,CA GB,5,5,CA,GB
10,1975.0,CA JP,2,2,CA,JP


In [194]:
for i in range(len(df_con1975)):
    if df_con1975['P2'][i] + " " + df_con1975['P1'][i] in df_con1975['Partner'].values:
        df_con1975.drop(df_con1975.index[[i]])
print df_con1975

      Year Partner  Importer  Exporter  P1  P2
0   1975.0   BE CA         2         2  BE  CA
1   1975.0   BE CH         2         2  BE  CH
2   1975.0   BM CR         1         1  BM  CR
3   1975.0   BW CH         1         1  BW  CH
4   1975.0   CA BE         2         2  CA  BE
5   1975.0   CA CD         1         1  CA  CD
6   1975.0   CA CH         3         3  CA  CH
7   1975.0   CA DE         7         7  CA  DE
8   1975.0   CA DK         1         1  CA  DK
9   1975.0   CA GB         5         5  CA  GB
10  1975.0   CA JP         2         2  CA  JP
11  1975.0   CA NZ         1         1  CA  NZ
12  1975.0   CA US        11        11  CA  US
13  1975.0   CD CA         1         1  CD  CA
14  1975.0   CH BE         2         2  CH  BE
15  1975.0   CH BW         1         1  CH  BW
16  1975.0   CH CA         3         3  CH  CA
17  1975.0   CH DE        27        27  CH  DE
18  1975.0   CH FI         1         1  CH  FI
19  1975.0   CH FR        12        12  CH  FR
20  1975.0   

,Year,Partner,Importer,Exporter,P1,P2
0,1975.0,BE CA,2,2,BE,CA
1,1975.0,BE CH,2,2,BE,CH
2,1975.0,BM CR,1,1,BM,CR
3,1975.0,BW CH,1,1,BW,CH
4,1975.0,CA BE,2,2,CA,BE
5,1975.0,CA CD,1,1,CA,CD
6,1975.0,CA CH,3,3,CA,CH
7,1975.0,CA DE,7,7,CA,DE
8,1975.0,CA DK,1,1,CA,DK
9,1975.0,CA GB,5,5,CA,GB
